<div style="text-align: center; font-weight: bold; font-size: 300%"> PC 9 - MAP412</div>                                                        <br />      
<div style="text-align: center; font-size: 150%">École Polytechnique, MAP412 - 11 novembre 2019</div><br />  
<div style="text-align: center; font-size: 120%">Paul CALOT</div>

In [1]:
import numpy as np

from scipy.sparse import diags, csc_matrix, csr_matrix
from scipy.sparse import eye
from scipy.sparse.linalg import spsolve

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column, row

output_notebook(hide_banner=True)

# Résolution numérique d'EDP paraboliques

In [2]:
from time import time
carac=[] 

# on va comparer rapidement les temps d'execution schema explicite - implicite.

## Equation de la chaleur

Soient $\nu>0$ un coefficient de diffusion, un intervalle $\Omega=]0,L[$, $L>0$, et $u^0\in C^0(\Omega)$ une donnée initiale. On considère l'équation de la chaleur :

\begin{equation*}
\left\{
\begin{aligned}
  &\displaystyle\frac{\partial u}{\partial t}(t,x)-\nu\frac{\partial^2 u}{\partial x^2}(t,x) = 0, \quad &\forall~(t,x) \in\mathbb{R}_+^*\times\Omega, \\
  &\frac{\partial u}{\partial x}(t,0) = 0 = \frac{\partial u}{\partial x}(t,L )\quad &\forall~t\in\mathbb{R}_+^*, \\
  &u(x,0) = u^0(x), \quad &\forall x\in\Omega.
\end{aligned}
\right.
\end{equation*}
  

Pour la discrétisation en temps, on fixe un temps final $T>0$, puis on considère :
\begin{equation*}
    N\in\mathbb{R}^*, \qquad \Delta t = \frac{T}{N} \quad \text{et} \quad t^n = n\Delta t,\ n = 0, \dots, N.
\end{equation*}


Pour la discrétisation en espace, on considère :
\begin{equation*} J\in\mathbb{N}^*, \qquad \Delta x = \frac{L}{J+1} \quad\text{et}\quad x_j = j\Delta x,\ j = -1, \dots, J+2. \end{equation*}

On note $U^0_j = u^0(x_j)\ j = 0, \dots, J+1$, et on va chercher à construire de manière itérative une suite 
$U^n = \left(U^n_j\right)_{0\leq j \leq J+1}$ telle que $U^n_j$ approche $u(t^n,x_j)$.

Pour les questions d'implémentation, on prendra
\begin{equation*}
\nu = 1,\quad L=10,\quad J=100, \quad u^0(x) = 2 - \cos \frac{2\pi x}{L},
\end{equation*}

### Schéma explicite en temps

On considère le schéma "différences finies" explicite en temps donné par :

\begin{equation*}
\frac{U^{n+1}_j - U^n_j}{\Delta t} - \nu \frac{U^n_{j-1}-2U^n_j+U^n_{j+1}}{(\Delta x)^2} = 0 \qquad \forall~n\geq 0,\quad \forall~0 \leq j\leq J+1.
\end{equation*}

Afin d'approcher les conditions aux bords, on pourra utiliser les conventions :

\begin{equation}
  \label{Neumann1} 
  \frac{U_0^n - U^n_{-1}}{\Delta x} = 0 \quad\text{et}\quad \frac{U^n_{J+2} - U^n_{J+1}}{\Delta x} = 0, 
\end{equation}

ou

\begin{equation}
  \label{Neumann2} 
  \frac{U_1^n - U^n_{-1}}{2\Delta x} = 0 \quad\text{et}\quad \frac{U^n_{J+2} - U^n_{J}}{2\Delta x} = 0. 
\end{equation}



**Implémenter le schéma "différences finies" explicite en temps pour les deux conventions  $(1)$ et $(2)$.**  

In [3]:
# definition de la matrice du système U_(n+1)=Aexpl U_(n) pour les conventions (1) et (2) :

def Aexpl(J,alpha,i):
    """
    J : nombre d'intervalles pour la discrétisation spatiale. 
    alpha = nu (dt / dx^2)
    i : convention choisie.
    """
    
    
    # Definition prealable des diagonales
    d0=(1-2*alpha)*np.ones(J+2)
    d1=alpha*np.ones(J+1)
    d2=alpha*np.ones(J+1) # attention aux nombres de points...
    
    if i==1: # la convention choisie est (1)
        d0[0],d0[-1]=1-alpha,1-alpha
        
    else : # (2)
        d1[0]=2*alpha
        d2[-1]=2*alpha
        
    # stockage et definition par diagonale.

    A=diags([d0,d1,d2],[0,1,-1])
    
    return A

In [4]:
def diffFinies_expl(i,u0,T,dt,L=10,J=1000,nu=1):
    """
        i : convention choisie
        u0 : fonction permettant d'obtenir les conditions initiales.
        T : intervalle temporel d'étude.
        dt : pas de temps. N=T/dt
        L : intervalle spatiale d'étude.
        J : nombre d'int²rvalles pour la discrétisation spatiale (J+1 points).
        nu : coefficient de diffusion thermique.
    """
    # définition des constantes utilisées pour le problème :
    N=int(T/dt) 
    dx=L/(J+1)
    
    alpha = nu*dt/(dx*dx)
    
    # initialisation du schéma :
    X=np.array([j*dx for j in range(J+2)])
    U0=u0(X)
    A=Aexpl(J,alpha,i) # format creux
    
    # stockage des Uk à l'instant tn
    U_liste=np.empty((N+1,J+2))
    U_liste[0]=U0
    
    for k in range(1,N+1):
        U_liste[k]=A.dot(U_liste[k-1]) # A.dot: methode d'objet associee à A sous sa representation creuse
    
    return U_liste

**Pour chaque traitement des conditions aux bords $(1)$ et $(2)$, utiliser $\Delta t= 0.004$ et afficher les solutions obtenues pour $T=0$, $T=1$, $T=5$ et $T=20$  (sur un même graphique, avec différentes couleurs). Afficher également la masse $\sum_{j=0}^{J+1} U^{n+1}_j$ en fonction de $n$, et l'énergie $\left\Vert U^n \right\Vert_{L^2}$ en fonction de $n$. Répéter ces expériences avec $\Delta t= 0.005$ et $T=0$, $T=1$, $T=2$ et $T=5$. Calculer $\alpha=\frac{\nu \Delta t}{\Delta x^2}$ dans les deux cas, et commenter les différents résultats obtenus.**

In [5]:
# quelques fonctions utiles pour la suite : 

def Masse(U,dx):
    S=0
    for j in range(len(U)): # len(Uk)=J+2
        S+=U[j]
    return S*dx

def Norme_L2(U,dx): 
    Norme_U=np.linalg.norm(U)
    return dx*Norme_U*Norme_U
    # rdx : racine carré de dx            

In [6]:
def afficher(i,dt,T_liste,scheme,L=10,J=1000,cols=['orange','firebrick','darkgreen','blue','olive','lavender']):
    """
    i = {1,2}: conditions aux bords.
    scheme : schema à utiliser (implicite ou explicite)
    dt (int) : pas de temps
    T_liste (ndarray) : trie par ordre croissant - temps auxquels on représente le système.
    """
    
    # constantes : 
    dx=L/J
    J_liste=np.linspace(0,L,J+2)
    
    
    T=T_liste[-1]
    N=int(T/dt) 
    N_liste=np.linspace(0,T,N+1)
    
    k=2*np.pi/L
    
    # calcul jusqu'au temps T = 20.
    t1=time()
    U_liste=scheme(i,lambda x: 2-2*np.cos(k*x),T,dt,L=L,J=J) 
    t2=time()
    
    
    # définition des figures : 
         # on centre la masse...
    M0=Masse(U_liste[0],dx) # pour centrer la masse - facilite la lecture.
    fig1 = figure(width=800, height=500,title="Differentes solutions pour dt="+str(dt))
    fig2 = figure(width=450, height=300,title="Masse centrée en 0 en fonction de n, M0 = "+str(round(M0,3)))
    fig3 = figure(width=450, height=300,title="Energie en fonction de n")
    
    # axes
    fig1.xaxis.axis_label = "x"
    fig1.yaxis.axis_label = "U(x)"
    fig2.xaxis.axis_label = "t"
    fig2.yaxis.axis_label = "Masse(t) - Masse(0)"
    fig3.xaxis.axis_label = "t"
    fig3.yaxis.axis_label = "Energie(t)"
    
    
    # tracé pour les T considérés
    for j in range(len(T_liste)):
        fig1.line(J_liste,U_liste[int(T_liste[j]/T * N)],line_width=1,line_color=cols[j],legend="T = "+str(T_liste[j]))
    
        # tracé de la masse
    MasseU_liste=[Masse(U_liste[t],dx)-M0 for t in range(N+1)]
    
    fig2.line(N_liste,MasseU_liste,line_width=1,line_color=cols[j],legend="T = "+str(T))
    
        # tracé de l'énergie
    NormeU_liste=np.array([Norme_L2(U_liste[t],dx) for t in range(N+1)])
        
    fig3.line(N_liste,NormeU_liste,line_width=1,line_color=cols[j],legend="T = "+str(T))
    
    print(" --------------------------------------------------------------------------- ")
    print("")
    print("La valeur de alpha est : ", dt/(dx*dx))
    print("")
        
    fig1.legend.location = "top_left"
    fig2.legend.location = "bottom_center"
    fig3.legend.location = "top_center"

    show(fig1)
    show(row(fig2,fig3))
    

    return((t2-t1)*dt/T) # on divise par N afin de comptabiliser uniquement le temps moyen par itération.

In [7]:
 # Présentation des résultats : 
    # définition des paramètres utilités :
dt=[0.004,0.005] 
T_liste=np.array([[0,1,5,20],[0,1,2,5]])

    # Calculs numériques
        # dt=0.004
        
# on prend J=100, en accord le TD, pour J=1000, le alpha est trop grand pour assurer les conditions de stabilité.
                  
carac.append(afficher(1,dt[0],T_liste[0],diffFinies_expl,J=100))
                  
afficher(2,dt[0],T_liste[0],diffFinies_expl,J=100)

        # dt=0.005
    
afficher(1,dt[1],T_liste[1],diffFinies_expl,J=100)
                  
afficher(2,dt[1],T_liste[1],diffFinies_expl,J=100)

 --------------------------------------------------------------------------- 

La valeur de alpha est :  0.3999999999999999



 --------------------------------------------------------------------------- 

La valeur de alpha est :  0.3999999999999999



 --------------------------------------------------------------------------- 

La valeur de alpha est :  0.4999999999999999



 --------------------------------------------------------------------------- 

La valeur de alpha est :  0.4999999999999999



0.0

Les calculs réalisés à la main donne :
<ul> <li> $\alpha = 0.4$ pour $\Delta t = 0.004$.
    <li> $\alpha = 0.5$ pour $\Delta t = 0.005$. 
</ul>
La stabilité est théoriquement assurée pour $\alpha \leq \frac{1}{2}$ (et la convergence pour $\alpha < \frac{1}{2}$). 

In [8]:
import pandas
l = [ {"conditions aux bords":"Non centré (1)", "consistance": "O(\u0394 t + \u0394 x)", "Conservation Masse":"Oui", "observations":"DV pour \u03B1 = 0.5, CV sinon."},
     {"conditions aux bords":"Centré (2)", "consistance": "O(\u0394 t + \u0394 x²)", "Conservation Masse":"Non", "observations":"Sol bornée pour \u03B1 = 0.5, CV sinon."}]
df = pandas.DataFrame(l)
df

,conditions aux bords,consistance,Conservation Masse,observations
0,Non centré (1),O(Δ t + Δ x),Oui,"DV pour α = 0.5, CV sinon."
1,Centré (2),O(Δ t + Δ x²),Non,"Sol bornée pour α = 0.5, CV sinon."


Plusieurs observations :
<ul>
    <li> Le schéma convergence pour (1) vers la solution constante égale à 1 (logique, pas de terme source dans l'équation de la chaleur). 
    <li> On observe la conservation de la masse pour (1), aux erreurs machines près, mais pas pour (2). 
    <li> On a bien la décroissance de l'énergie (sauf pour $\alpha = 0.5$). 
    <li> Du point de vue théorique, la stabilité du schéma pour $\alpha = 0.5$ n'est pas confortée par l'expérience. En effet, même si on n'a pas de convergence, on devrait retrouver une solution bornée. Or :
        <ul> <li> Pour les conditions (1), la solution ainsi que la masse et l'énergie du système divergent. </li> 
             <li> Pour les conditions (2), la solution ainsi que la masse et l'énergie du système semblent bornés ou alors divergent beaucoup plus lentement.</li>
        </ul> 
Cela peut s'expliquer par deux effets : 
         <ul> <li> Les erreurs machines mettent à mal la stabilité du schéma et provoquent la divergence pour les deux conditions aux bords. </li> 
              <li> La meilleure consistance de (2) par rapport à (1) en x (en $\Delta x^2$ pour (2) contre $\Delta x$ pour (1) ) peut justifier des meilleurs résultats obtenus pour (2) par rapport à (1). </li>
        </ul>  
</ul>
  

### Schéma implicite en temps

On considère le schéma "différences finies" implicite en temps donné par :

\begin{equation*}
\label{implicite}
\frac{U^{n+1}_j - U^n_j}{\Delta t} - \nu \frac{U^{n+1}_{j-1}-2U^{n+1}_j+U^{n+1}_{j+1}}{(\Delta x)^2} = 0 \qquad \forall~n\geq 1,\ \forall~ 0\leq j\leq J+1.
\end{equation*}

**Implémenter le schéma "différences finies" implicite en temps pour les deux conventions $(1)$ et $(2)$.**

In [9]:
# définition de la matrice du système U_(n)=Aimpl U_(n+1) pour les conventions (1) et (2) :

def Aimpl(J,alpha,i):
    """
    J : nombre d'intervalles pour la discrétisation spatiale. 
    alpha = nu (dt / dx^2)
    i : convention choisie.
    """
    
    
    # Définition préalable des diagonales
    d0=(1+2*alpha)*np.ones(J+2)
    d1=-alpha*np.ones(J+1) # diagonale supérieure
    d2=-alpha*np.ones(J+1) # attention aux nombres de points...
    
    if i==1: # la convention choisie est (1)
        d0[0],d0[-1]=1+alpha,1+alpha
        
    else : # (2)
        d1[0]=-2*alpha
        d2[-1]=-2*alpha
        
    # stockage et définition par diagonale.

    A=diags([d0,d1,d2],[0,1,-1])
    
    
    return A


In [10]:
def diffFinies_impl(i,u0,T,dt,L=10,J=1000,nu=1):
    """
        i : convention choisie
        u0 : fonction permettant d'obtenir les conditions initiales.
        T : intervalle temporel d'étude.
        N : nombre d'intervalles pour la discrétisation temporelle.
        L : intervalle spatiale d'étude.
        J : nombre d'intervalles pour la discrétisation spatiale (J+1 points).
        nu : coefficient de diffusion thermique.
    """
    # définition des constantes utilisées pour le problème :
    dx=L/(J+1)
    N=int(T/dt) 
    alpha = (nu/(dx*dx))*dt
    
    # initialisation du schéma :
    X=np.array([j*dx for j in range(J+2)])
    U0=u0(X)    # la fonction doit supporter les méthodes numpy. 
    
    A=Aimpl(J,alpha,i) # format creux

    # stockage des Uk à l'instant tn
    U_liste=np.empty((N+1,J+2))
    U_liste[0]=U0
    
    
    for k in range(1,N+1):
        U_liste[k]=spsolve(A,U_liste[k-1])
    
    return U_liste

**Pour chaque traitement des conditions aux bords $(1)$ et $(2)$, utiliser $\Delta t= 0.01$ et afficher les solutions obtenues pour $T=0$, $T=1$, $T=5$ et $T=20$  (sur un même graphique, avec différentes couleurs). Afficher également la masse $\sum_{j=0}^{J+1} U^{n+1}_j$ en fonction de $n$, et l'énergie $\left\Vert U^n \right\Vert_{L^2}$ en fonction de $n$. Commenter les différents résultats obtenus et discuter des avantages et des inconvénients des schémas explicites et implicites pour ce problème.**

In [11]:
carac.append(afficher(1,0.01,T_liste[0],diffFinies_impl,J=100))

afficher(2,0.01,T_liste[0],diffFinies_impl,J=100)

C:\Users\Paul\Anaconda3\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:133: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)


 --------------------------------------------------------------------------- 

La valeur de alpha est :  0.9999999999999998



 --------------------------------------------------------------------------- 

La valeur de alpha est :  0.9999999999999998



0.00035148000717163085

On obtient des résultats qui confortent du point de vue physique ce qui se passe précédemment (conservation de la masse, aux erreurs machines près, pour (1), décroissance de l'énergie pour tous, convergence vers une solution constante égale à $1$), cependant le schéma implicite est inconditionnellement stable.
On peut donc trouver deux différences principales entre le schéma implicite et le schéma explicite : 
<ul>
<li> Le schéma implicite est inconditionnellement stable contrairement au schéma implicite pour lequel la condition de stabilité est $\alpha \leq \frac{1}{2}$.
<li> Le schéma explicite est à l'inverse beaucoup plus simple à mettre en place (et est probablement également moins coûteux en calcul, puisqu'on a pas besoin d'inverser la matrice de discrétisation, cf. ci-après pour une certaine idée). 
</ul>
Ci-après un résumé de ce qu'on vient de voir (théoriquement d'abord puis empiriquement):

In [12]:

l = [ {"schéma":"explicite", "conditions de stabilité":"\u03B1 < 0.5", "conditions aux bords":"Non centré (1)  /  Centré (2)", "consistance": "O(\u0394 t + \u0394 x) / O(\u0394 t + \u0394 x²) ", "Conservation Masse":"Oui  /  Non"},
     {"schéma":"implicite", "conditions de stabilité":"Inconditionnel", "conditions aux bords":"Non centré (1)  /  Centré (2)", "consistance": "O(\u0394 t + \u0394 x) / O(\u0394 t + \u0394 x²) ", "Conservation Masse":"Oui  /  Non"}]
df = pandas.DataFrame(l)
df

,schéma,conditions de stabilité,conditions aux bords,consistance,Conservation Masse
0,explicite,α < 0.5,Non centré (1) / Centré (2),O(Δ t + Δ x) / O(Δ t + Δ x²),Oui / Non
1,implicite,Inconditionnel,Non centré (1) / Centré (2),O(Δ t + Δ x) / O(Δ t + Δ x²),Oui / Non


In [13]:
print("Le temps moyen par itération pour le schema explicite est ", carac[0], "contre ", carac[1]," pour le schema implicite.")

Le temps moyen par itération pour le schema explicite est  9.372901916503905e-06 contre  0.000437420129776001  pour le schema implicite.


## Étude d'une onde progressive

On considère l'équation de Nagumo :

\begin{equation*}
  \label{Nagumo_db}
  \left\{
  \begin{aligned}
  &\displaystyle\frac{\partial u}{\partial t}(t,x)-\nu\frac{\partial^2 u}{\partial x^2}(t,x) = g(u(t,x)), \quad &\forall~(t,x) \in\mathbb{R}_+^*\times\Omega,  \\
  &\frac{\partial u}{\partial x}(t,0) = 0 = \frac{\partial u}{\partial x}(t,L )\quad &\forall~t\in\mathbb{R}_+^*, \\
  &u(x,0) = u^0(x), \quad &\forall x\in\Omega,
  \end{aligned}
  \right.
\end{equation*}

où $g:\mathbb{R} \to \mathbb{R}$ est donnée par :

\begin{equation*}
\label{g}
g(u) = ku^2(1-u).
\end{equation*}

Pour les simulations, on prendra
\begin{equation*}
\nu=0.1,\quad L=100,\quad k=10,\quad \text{et}\quad u^0(x) =
\left\{
\begin{aligned}
& 1,\quad & x\leq \frac{L}{10} \\
& 0,\quad & x >\frac{L}{10}.
\end{aligned}
\right.
\end{equation*}

On utilisera les mêmes discrétisations spatiales et temporelles que dans l'exercice précédent, avec $J=1000$ ($\Delta x = \frac{L}{J+1}$).

### Schéma totalement explicite

On considère le schéma donné par :

\begin{equation*}
\frac{U^{n+1}_j - U^n_j}{\Delta t} - \nu \frac{U^n_{j-1}-2U^n_j+U^n_{j+1}}{(\Delta x)^2} = g(U^n_j) \qquad \forall~n\geq 1,\ \forall~ 0\leq j\leq J+1,
\end{equation*}

**Implémenter ce schéma en approchant les conditions aux bords avec les conventions (1)**

**Afficher les solutions obtenues à $T=0$, $T=20$, $T=40$ et $T=60$, avec $\Delta t = 0.025$ (sur un même graphique, avec différentes couleurs). Utiliser maintenant $\Delta t =0.045$ et afficher la solution obtenue à $T\approx 20$. Commenter *(on pourra calculer $\alpha = \frac{\nu\Delta t}{\Delta x}$)*.**

### Schéma semi-implicite.

On considère le schéma donné par :

\begin{equation*}
\frac{U^{n+1}_j - U^n_j}{\Delta t} - \nu \frac{U^{n+1}_{j-1}-2U^{n+1}_j+U^{n+1}_{j+1}}{(\Delta x)^2} = g(U^n_j) \qquad \forall~n\geq 1,\ \forall~ 0\leq j\leq J+1,
\end{equation*}

**Implémenter ce schéma en approchant les conditions aux bords avec les conventions (1)**

**afficher les solutions obtenues à $T=0$, $T=20$, $T=40$ et $T=60$, avec $\Delta t = 0.045$ (sur un même graphique, avec différentes couleurs). Utiliser maintenant $\Delta t =0.4$ et afficher la solution obtenue à $T\approx 20$. Commenter.**

### Schéma totalement implicite (bonus).

On considère le schéma donné par :

\begin{equation*}
\frac{U^{n+1}_j - U^n_j}{\Delta t} - \nu \frac{U^{n+1}_{j-1}-2U^{n+1}_j+U^{n+1}_{j+1}}{(\Delta x)^2} = g(U^{n+1}_j) \qquad \forall~n\geq 1,\ \forall~ 0\leq j\leq J+1,
\end{equation*}

**Bonus : Implémenter ce schéma en approchant les conditions aux bords avec les conventions (1)**

**Bonus : Afficher les solutions obtenues à $T=0$, $T=20$, $T=40$ et $T=60$, avec $\Delta t = 0.4$ (sur un même graphique, avec différentes couleurs). Comparer avec les solutions obtenues, pour le même schéma, avec $\Delta t = 0.04$.**